In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Activation, Layer

In [60]:
sample1, sample2 = [0, 1, 0], [1, 0, 1]

sample1_fea_embed = tf.constant(
    [[ 0.01288019,  0.03272445, -0.01649339],
     [ -0.01770421, -0.00483744,  0.00950954],
     [ 0.04772587,  0.04583821, -0.0478565]]
)
sample2_fea_embed = tf.constant(
    [[0.04020175, -0.00628924,  0.00319413],
     [0.0129966,  -0.02178497,  0.04210693],
     [-0.00487811, -0.01407316, -0.02572885]]
)

fm_dense_sample1 = tf.constant([0.01222704, -0.04932194, 0.00101062])
fm_dense_sample2 = tf.constant([0.04848984, -0.01674583, 0.00623454])

fm_bias = tf.constant([-0.084661])

deep_dense_1 = tf.constant(
[[-0.06075704, -0.45923695,  0.1717745 ],
 [-0.6454746,  -0.00456971,  0.36452907],
 [ 0.33800882,  0.36106378, -0.64581275],
 [ 0.26727337, -0.03098249,  0.6380921 ],
 [-0.3135201,   0.33219272,  0.23493767],
 [-0.04501891, -0.3936964,  -0.19730657],
 [-0.5929896,   0.5451148,   0.63756806],
 [ 0.50880164, -0.16037196, -0.37451488],
 [ 0.47448808,  0.37647945, -0.16552669]]
)
deep_bias_1 = tf.constant([0., 0., 0.])
deep_dense_2 = tf.constant(
[[ 0.88652265],
 [-0.04007936],
 [ 0.16886973]]
)
deep_bias_2 = tf.constant([0.])

In [77]:
# fm part
sample1_one_order = \
    tf.reduce_sum(fm_dense_sample1)
print("sample1 one order")
print(sample1_one_order.numpy())
print()

sample2_one_order = \
    tf.reduce_sum(fm_dense_sample2)
print("sample2 one order")
print(sample2_one_order.numpy())
print()

sample1_two_order = \
    tf.matmul(tf.expand_dims(sample1_fea_embed[0], axis=0), tf.expand_dims(sample1_fea_embed[1], axis=-1)) + \
    tf.matmul(tf.expand_dims(sample1_fea_embed[0], axis=0), tf.expand_dims(sample1_fea_embed[2], axis=-1)) + \
    tf.matmul(tf.expand_dims(sample1_fea_embed[1], axis=0), tf.expand_dims(sample1_fea_embed[2], axis=-1))
print("sample1 two order")
print(tf.squeeze(sample1_two_order).numpy())
print()

sample2_two_order = \
    tf.matmul(tf.expand_dims(sample2_fea_embed[0], axis=0), tf.expand_dims(sample2_fea_embed[1], axis=-1)) + \
    tf.matmul(tf.expand_dims(sample2_fea_embed[0], axis=0), tf.expand_dims(sample2_fea_embed[2], axis=-1)) + \
    tf.matmul(tf.expand_dims(sample2_fea_embed[1], axis=0), tf.expand_dims(sample2_fea_embed[2], axis=-1))
print("sample2 two order")
print(tf.squeeze(sample2_two_order).numpy())
print()

print("sample1 fm out")
sample1_fm_out = sample1_one_order + sample1_two_order + fm_bias
print(sample1_fm_out.numpy())

print("sample2 fm out")
sample2_fm_out = sample2_one_order + sample2_two_order + fm_bias
print(sample2_fm_out.numpy())

sample1 one order
-0.03608428

sample2 one order
0.037978552

sample1 two order
0.0008391021

sample2 two order
-0.0002359667

sample1 fm out
[[-0.11990617]]
sample2 fm out
[[-0.04691841]]


In [62]:
# deep part
sample1_fea_embed_concated = tf.expand_dims(tf.reshape(sample1_fea_embed, (-1, )), 0)
sample2_fea_embed_concated = tf.expand_dims(tf.reshape(sample2_fea_embed, (-1, )), 0)

In [76]:
sample1_deep1_out = tf.matmul(sample1_fea_embed_concated, deep_dense_1) + deep_bias_1
sample1_deep1_out = tf.nn.relu(sample1_deep1_out)
print("sample1 deep1 out")
print(sample1_deep1_out.numpy())

sample1_deep2_out = tf.matmul(sample1_deep1_out, deep_dense_2) + deep_bias_2
sample1_deep2_out = tf.nn.relu(sample1_deep2_out)
print("sample1 deep2 out")
print(sample1_deep2_out.numpy())

sample2_deep1_out = tf.matmul(sample2_fea_embed_concated, deep_dense_1) + deep_bias_1
sample2_deep1_out = tf.nn.relu(sample2_deep1_out)
print("sample2 deep1 out")
print(sample2_deep1_out.numpy())

sample2_deep2_out = tf.matmul(sample2_deep1_out, deep_dense_2) + deep_bias_2
sample2_deep2_out = tf.nn.relu(sample2_deep2_out)
print("sample2 deep2 out")
print(sample2_deep2_out.numpy())




sample1 deep1 out
[[0.         0.         0.03166636]]
sample1 deep2 out
[[0.00534749]]
sample2 deep1 out
[[0.         0.         0.00383645]]
sample2 deep2 out
[[0.00064786]]


In [78]:
# final sigmoid out
sample1_pred = tf.nn.sigmoid(sample1_fm_out+sample1_deep2_out)
print("sample1 pred")
print(sample1_pred)

sample2_pred = tf.nn.sigmoid(sample2_fm_out+sample2_deep2_out)
print("sample2 pred")
print(sample2_pred)

sample1 pred
tf.Tensor([[0.47139162]], shape=(1, 1), dtype=float32)
sample2 pred
tf.Tensor([[0.48843443]], shape=(1, 1), dtype=float32)
